<div class="contentcontainer med left" style="margin-left: -50px;">
<dl class="dl-horizontal">
  <dt>Title</dt> <dd> QuadMesh Element</dd>
  <dt>Dependencies</dt> <dd>Bokeh</dd>
  <dt>Backends</dt> <dd><a href='./QuadMesh.ipynb'>Bokeh</a></dd> <dd><a href='../matplotlib/QuadMesh.ipynb'>Matplotlib</a></dd>
</dl>
</div>

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

The basic ``QuadMesh`` is a 2D grid of bins specified as x-/y-values specifying a regular sampling or edges, with arbitrary sampling and an associated 2D array containing the bin values. Here is a simple ``QuadMesh`` with logarithmic sampling along the 'x' dimensions:

In [ ]:
n = 8  # Number of bins in each direction
xs = np.logspace(1, 3, n)
ys = np.linspace(1, 10, n)
zs = np.arange((n-1)**2).reshape(n-1, n-1)
print('Shape of x-coordinates:', xs.shape)
print('Shape of y-coordinates:', ys.shape)
print('Shape of value array:', zs.shape)
hv.QuadMesh((xs, ys, zs))

The coordinate system of a ``QuadMesh`` is defined by the bin edges, therefore any index falling into a binned region will return the appropriate value. ``QuadMesh`` supports inspection of values using the Bokeh hover tools and because the bin edges have continuous values, you can use non-linear axes such as log axes:

In [ ]:
%%opts QuadMesh [tools=['hover'] xticks=[10, 100,1000]] QuadMesh.LogScale [logx=True]
hv.QuadMesh((xs, ys, zs), group='LinearScale') + hv.QuadMesh((xs, ys, zs), group='LogScale')

Unlike ``Image`` objects, slices must be inclusive of the bin edges but otherwise the slicing semantics are the same. The reason for this difference is that ``QuadMesh`` is really a two-dimensional histogram and when slicing, you only want to see the bins that fall within the specified slice ranges.

In the next example, we specify a slice along the x- and y-axis to extract the lower corner and we set the z-dimension range to maintain the full color range of the colormap:

In [ ]:
qmesh = hv.QuadMesh((xs, ys, zs))
qmesh[20:400, :8].redim.range(z=qmesh.range('z'))

As ``QuadMesh`` supports the Bokeh hover tool while ``Image`` does not, we can get use a simple trick to enable hover information: overlay the image with an invisible ``QuadMesh`` cast from the same image. We can set the ``QuadMesh`` alpha to zero while keeping an opaque hover line color:

In [ ]:
%%opts QuadMesh [tools=['hover']] (alpha=0 hover_line_alpha=1 hover_line_color='black')
img = hv.Image(np.random.rand(10,10))
img * hv.QuadMesh(img)

Now when you hover over this visualization, you can see the sample value as well as an outline of the selected bin.

For full documentation and the available style and plot options, use ``hv.help(hv.QuadMesh).``

## Irregular meshes

In addition to axis aligned meshes like those we worked with above, a ``QuadMesh`` may also be used to represent irregular or unstructured meshes. In this example we will create an irregular mesh consisting of 2D X, Y and Z arrays defining the position and value of each simplex in the mesh:

In [ ]:
n=20
coords = np.linspace(-1.5,1.5,n)
X,Y = np.meshgrid(coords, coords);
Qx = np.cos(Y) - np.cos(X)
Qy = np.sin(Y) + np.sin(X)
Z = np.sqrt(X**2 + Y**2)

print('Shape of x-coordinates:', Qx.shape)
print('Shape of y-coordinates:', Qy.shape)
print('Shape of value array:', Z.shape)

qmesh = hv.QuadMesh((Qx, Qy, Z))
qmesh

To illustrate irregular meshes a bit further we will randomly jitter the mesh coordinates along both dimensions, demonstrating that ``QuadMesh`` may be used to represent completely arbitrary meshes. It may also be used to represent overlapping meshes, however the behavior during slicing and other operations may not be well defined in such cases.

In [ ]:
np.random.seed(13)
xs, ys = np.meshgrid(np.linspace(-20, 20, 10), np.linspace(0, 30, 8))
xs += xs/10 + np.random.rand(*xs.shape)*4
ys += ys/10 + np.random.rand(*ys.shape)*4

zs = np.arange(80).reshape(8, 10)
hv.QuadMesh((xs, ys, zs))